In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
from utils import *

In [3]:
adj, adj_n, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus('r8')

In [4]:
adj.shape

(15362, 15362)

In [5]:
adj2 = adj.dot(adj)

In [6]:
adj2.shape

(15362, 15362)

In [12]:
adj3 = adj2.dot(adj)

In [13]:
adj3.shape

(15362, 15362)

In [15]:
with open('R8_adj_1.npy', 'wb') as f:
    np.save(f, adj)

In [16]:
with open('R8_adj_2.npy', 'wb') as f:
    np.save(f, adj2)

In [17]:
with open('R8_adj_3.npy', 'wb') as f:
    np.save(f, adj3)

In [18]:
adj4 = adj3.dot(adj)

In [19]:
adj4.shape

(15362, 15362)

In [20]:
with open('R8_adj_4.npy', 'wb') as f:
    np.save(f, adj4)